In [4]:
# ============================================================
# CELL 1: INSTALL VISUALIZATION DEPENDENCIES
# ============================================================

print("=" * 60)
print("INSTALLING VISUALIZATION DEPENDENCIES")
print("=" * 60)

print("\nInstalling required packages...")
print("This may take 1-2 minutes...\n")

# Install seaborn and other visualization packages
!pip install -q seaborn
!pip install -q plotly
!pip install -q kaleido  # For saving plotly figures

print("\n✓ Visualization dependencies installed")
print("=" * 60)

INSTALLING VISUALIZATION DEPENDENCIES

Installing required packages...
This may take 1-2 minutes...


✓ Visualization dependencies installed


In [5]:
# ============================================================
# CELL 2: IMPORT LIBRARIES
# ============================================================

print("\n" + "=" * 60)
print("NOTEBOOK 5: COMPREHENSIVE ANALYSIS & REPORTING")
print("=" * 60)

import torch
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
from collections import Counter

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("\n✓ All libraries imported successfully")
print("=" * 60)


NOTEBOOK 5: COMPREHENSIVE ANALYSIS & REPORTING

✓ All libraries imported successfully


In [7]:
# ============================================================
# CELL 3: LOAD ALL PROJECT DATA
# ============================================================

print("\n" + "=" * 60)
print("LOADING PROJECT DATA")
print("=" * 60)

# Load personas
personas_file = Path("./personas_output/generated_personas_30.json")
if personas_file.exists():
    with open(personas_file, 'r') as f:
        personas = json.load(f)
    print(f"✓ Loaded {len(personas)} personas")
else:
    print("❌ Personas not found")
    personas = []

# Load test scenarios
scenarios_file = Path("./test_scenarios.json")
if scenarios_file.exists():
    with open(scenarios_file, 'r') as f:
        test_scenarios = json.load(f)
    print(f"✓ Loaded {len(test_scenarios)} test scenarios")
else:
    print("❌ Test scenarios not found")
    test_scenarios = []

# Load test results
results_file = Path("./test_results_output/usability_test_results.json")
if results_file.exists():
    with open(results_file, 'r') as f:
        test_results = json.load(f)
    print(f"✓ Loaded {len(test_results)} test results")
else:
    print("❌ Test results not found")
    test_results = []

# Load diversity analysis
diversity_file = Path("./personas_output/diversity_report.json")
if diversity_file.exists():
    with open(diversity_file, 'r') as f:
        diversity_data = json.load(f)
    print(f"✓ Loaded diversity analysis")
else:
    print("⚠️ Diversity analysis not found")
    diversity_data = {}

print("=" * 60)


LOADING PROJECT DATA
✓ Loaded 30 personas
✓ Loaded 150 test scenarios
✓ Loaded 50 test results
✓ Loaded diversity analysis


In [8]:
# ============================================================
# CELL 4: COMPREHENSIVE ANALYSIS FUNCTIONS
# ============================================================

print("\n" + "=" * 60)
print("ANALYSIS FUNCTIONS")
print("=" * 60)

def analyze_success_by_tech_level(test_results, personas):
    """Analyze success rates by tech proficiency level"""
    
    # Create persona lookup
    persona_dict = {p['id']: p for p in personas}
    
    # Collect data
    tech_level_data = {}
    
    for result in test_results:
        persona_id = result['persona_id']
        persona = persona_dict.get(persona_id)
        
        if not persona:
            continue
        
        tech_level = persona['tech_proficiency']
        
        if tech_level not in tech_level_data:
            tech_level_data[tech_level] = {
                'total': 0,
                'successful': 0,
                'times': []
            }
        
        tech_level_data[tech_level]['total'] += 1
        if result['task_completed']:
            tech_level_data[tech_level]['successful'] += 1
        tech_level_data[tech_level]['times'].append(result['time_seconds'])
    
    # Calculate statistics
    analysis = {}
    for tech_level, data in tech_level_data.items():
        success_rate = (data['successful'] / data['total'] * 100) if data['total'] > 0 else 0
        avg_time = np.mean(data['times']) if data['times'] else 0
        
        analysis[tech_level] = {
            'success_rate': success_rate,
            'avg_time_seconds': avg_time,
            'total_tests': data['total']
        }
    
    return analysis


def identify_critical_issues(test_results):
    """Identify most critical usability issues"""
    
    # Collect all issues
    all_issues = []
    high_severity_issues = []
    
    for result in test_results:
        issues = result.get('issues_encountered', [])
        all_issues.extend(issues)
        
        if result.get('severity') == 'high':
            high_severity_issues.extend(issues)
    
    # Count occurrences
    issue_counts = Counter(all_issues)
    critical_counts = Counter(high_severity_issues)
    
    # Calculate impact scores
    critical_issues = []
    for issue, count in issue_counts.most_common(10):
        impact_score = count / len(test_results) * 100
        critical_count = critical_counts.get(issue, 0)
        
        critical_issues.append({
            'issue': issue,
            'occurrences': count,
            'impact_percentage': impact_score,
            'critical_occurrences': critical_count,
            'priority': 'HIGH' if critical_count > 0 else 'MEDIUM'
        })
    
    return critical_issues


def analyze_persona_diversity(personas):
    """Analyze persona diversity metrics"""
    
    age_dist = Counter([p['age_range'] for p in personas])
    tech_dist = Counter([p['tech_proficiency'] for p in personas])
    user_type_dist = Counter([p['user_type'] for p in personas])
    
    diversity_metrics = {
        'age_distribution': dict(age_dist),
        'tech_distribution': dict(tech_dist),
        'user_type_distribution': dict(user_type_dist),
        'total_personas': len(personas)
    }
    
    return diversity_metrics


def calculate_roi_metrics(test_results):
    """Calculate return on investment metrics"""
    
    total_tests = len(test_results)
    
    # Traditional testing estimates
    traditional_time_per_test = 60  # 60 minutes per real user test
    traditional_cost_per_test = 100  # $100 per participant
    
    traditional_total_time = total_tests * traditional_time_per_test
    traditional_total_cost = total_tests * traditional_cost_per_test
    
    # AI testing actuals
    ai_total_time = sum(r['time_seconds'] for r in test_results) / 60  # Convert to minutes
    ai_cost_per_test = 0.10  # Estimated $0.10 per AI test
    ai_total_cost = total_tests * ai_cost_per_test
    
    # Calculate savings
    time_saved = traditional_total_time - ai_total_time
    cost_saved = traditional_total_cost - ai_total_cost
    
    roi = {
        'traditional_approach': {
            'time_minutes': traditional_total_time,
            'time_hours': traditional_total_time / 60,
            'cost_usd': traditional_total_cost
        },
        'ai_approach': {
            'time_minutes': ai_total_time,
            'time_hours': ai_total_time / 60,
            'cost_usd': ai_total_cost
        },
        'savings': {
            'time_minutes': time_saved,
            'time_hours': time_saved / 60,
            'time_percentage': (time_saved / traditional_total_time * 100),
            'cost_usd': cost_saved,
            'cost_percentage': (cost_saved / traditional_total_cost * 100)
        }
    }
    
    return roi


print("✓ Analysis functions loaded")
print("=" * 60)


ANALYSIS FUNCTIONS
✓ Analysis functions loaded


In [9]:
# ============================================================
# CELL 5: RUN COMPREHENSIVE ANALYSIS
# ============================================================

print("\n" + "=" * 60)
print("RUNNING COMPREHENSIVE ANALYSIS")
print("=" * 60)

if test_results and personas:
    
    print("\nAnalyzing test results...")
    
    # 1. Success by tech level
    tech_analysis = analyze_success_by_tech_level(test_results, personas)
    
    print("\n📊 Success Rates by Tech Proficiency:")
    for tech_level, stats in sorted(tech_analysis.items(), 
                                    key=lambda x: x[1]['success_rate'], 
                                    reverse=True):
        print(f"  • {tech_level}: {stats['success_rate']:.1f}% success "
              f"(avg time: {int(stats['avg_time_seconds']//60)}m {int(stats['avg_time_seconds']%60)}s)")
    
    # 2. Critical issues
    critical_issues = identify_critical_issues(test_results)
    
    print("\n🔍 Top 5 Critical Issues:")
    for i, issue in enumerate(critical_issues[:5], 1):
        print(f"  {i}. {issue['issue'][:70]}")
        print(f"     Impact: {issue['impact_percentage']:.1f}% of tests | Priority: {issue['priority']}")
    
    # 3. Persona diversity
    diversity_metrics = analyze_persona_diversity(personas)
    
    print(f"\n👥 Persona Diversity:")
    print(f"  • Total personas: {diversity_metrics['total_personas']}")
    print(f"  • Age ranges covered: {len(diversity_metrics['age_distribution'])}")
    print(f"  • Tech levels covered: {len(diversity_metrics['tech_distribution'])}")
    print(f"  • User types covered: {len(diversity_metrics['user_type_distribution'])}")
    
    # 4. ROI calculations
    roi = calculate_roi_metrics(test_results)
    
    print(f"\n💰 Return on Investment:")
    print(f"  Traditional Testing:")
    print(f"    • Time: {roi['traditional_approach']['time_hours']:.1f} hours")
    print(f"    • Cost: ${roi['traditional_approach']['cost_usd']:,.2f}")
    print(f"  AI-Powered Testing:")
    print(f"    • Time: {roi['ai_approach']['time_hours']:.1f} hours")
    print(f"    • Cost: ${roi['ai_approach']['cost_usd']:,.2f}")
    print(f"  SAVINGS:")
    print(f"    • Time saved: {roi['savings']['time_hours']:.1f} hours ({roi['savings']['time_percentage']:.1f}%)")
    print(f"    • Cost saved: ${roi['savings']['cost_usd']:,.2f} ({roi['savings']['cost_percentage']:.1f}%)")
    
    print("\n" + "=" * 60)

else:
    print("\n❌ Insufficient data for analysis")


RUNNING COMPREHENSIVE ANALYSIS

Analyzing test results...

📊 Success Rates by Tech Proficiency:
  • Expert: 100.0% success (avg time: 4m 37s)
  • Moderate: 90.0% success (avg time: 5m 33s)
  • High: 80.0% success (avg time: 4m 37s)
  • Advanced: 80.0% success (avg time: 4m 30s)
  • Limited: 50.0% success (avg time: 10m 44s)

🔍 Top 5 Critical Issues:
  1. Needed multiple attempts to complete action
     Impact: 12.0% of tests | Priority: HIGH
  2. Error message was confusing or unhelpful
     Impact: 10.0% of tests | Priority: HIGH
  3. Encountered expected pain point: Limited customization options
     Impact: 4.0% of tests | Priority: HIGH
  4. Encountered expected pain point: Confusing terminology
     Impact: 4.0% of tests | Priority: HIGH
  5. Struggled to find primary workflow completion elements
     Impact: 4.0% of tests | Priority: HIGH

👥 Persona Diversity:
  • Total personas: 30
  • Age ranges covered: 6
  • Tech levels covered: 7
  • User types covered: 10

💰 Return on Inve

In [10]:
# ============================================================
# CELL 6: CREATE VISUALIZATIONS
# ============================================================

print("\n" + "=" * 60)
print("CREATING VISUALIZATIONS")
print("=" * 60)

if test_results and personas:
    
    # Create output directory for visualizations
    viz_dir = Path("./visualizations_output")
    viz_dir.mkdir(exist_ok=True)
    
    # Visualization 1: Success Rate by Tech Level
    fig, ax = plt.subplots(figsize=(10, 6))
    
    tech_levels = list(tech_analysis.keys())
    success_rates = [tech_analysis[t]['success_rate'] for t in tech_levels]
    
    bars = ax.bar(tech_levels, success_rates, color='steelblue', alpha=0.8)
    ax.set_xlabel('Tech Proficiency Level', fontsize=12)
    ax.set_ylabel('Success Rate (%)', fontsize=12)
    ax.set_title('Task Success Rate by Tech Proficiency', fontsize=14, fontweight='bold')
    ax.set_ylim(0, 100)
    
    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%',
                ha='center', va='bottom')
    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(viz_dir / 'success_by_tech_level.png', dpi=300, bbox_inches='tight')
    print("✓ Created: success_by_tech_level.png")
    plt.close()
    
    # Visualization 2: Issue Frequency
    fig, ax = plt.subplots(figsize=(12, 8))
    
    top_issues = critical_issues[:10]
    issue_names = [i['issue'][:40] + '...' if len(i['issue']) > 40 else i['issue'] 
                   for i in top_issues]
    issue_counts = [i['occurrences'] for i in top_issues]
    colors = ['#d62728' if i['priority'] == 'HIGH' else '#1f77b4' for i in top_issues]
    
    bars = ax.barh(issue_names, issue_counts, color=colors, alpha=0.8)
    ax.set_xlabel('Number of Occurrences', fontsize=12)
    ax.set_title('Top 10 Usability Issues', fontsize=14, fontweight='bold')
    ax.invert_yaxis()
    
    # Add value labels
    for i, (bar, count) in enumerate(zip(bars, issue_counts)):
        ax.text(count, i, f' {count}', va='center')
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#d62728', alpha=0.8, label='High Priority'),
        Patch(facecolor='#1f77b4', alpha=0.8, label='Medium Priority')
    ]
    ax.legend(handles=legend_elements, loc='lower right')
    
    plt.tight_layout()
    plt.savefig(viz_dir / 'top_issues.png', dpi=300, bbox_inches='tight')
    print("✓ Created: top_issues.png")
    plt.close()
    
    # Visualization 3: Severity Distribution
    fig, ax = plt.subplots(figsize=(8, 8))
    
    severity_counts = Counter([r['severity'] for r in test_results])
    severities = list(severity_counts.keys())
    counts = list(severity_counts.values())
    colors_pie = ['#d62728', '#ff7f0e', '#2ca02c']
    
    ax.pie(counts, labels=severities, autopct='%1.1f%%', startangle=90,
           colors=colors_pie[:len(severities)], textprops={'fontsize': 12})
    ax.set_title('Test Results by Severity', fontsize=14, fontweight='bold')
    
    plt.savefig(viz_dir / 'severity_distribution.png', dpi=300, bbox_inches='tight')
    print("✓ Created: severity_distribution.png")
    plt.close()
    
    # Visualization 4: Completion Time Distribution
    fig, ax = plt.subplots(figsize=(10, 6))
    
    completion_times = [r['time_seconds']/60 for r in test_results]  # Convert to minutes
    
    ax.hist(completion_times, bins=20, color='steelblue', alpha=0.7, edgecolor='black')
    ax.set_xlabel('Completion Time (minutes)', fontsize=12)
    ax.set_ylabel('Number of Tests', fontsize=12)
    ax.set_title('Task Completion Time Distribution', fontsize=14, fontweight='bold')
    ax.axvline(np.mean(completion_times), color='red', linestyle='--', 
               linewidth=2, label=f'Mean: {np.mean(completion_times):.1f} min')
    ax.legend()
    
    plt.tight_layout()
    plt.savefig(viz_dir / 'completion_time_distribution.png', dpi=300, bbox_inches='tight')
    print("✓ Created: completion_time_distribution.png")
    plt.close()
    
    print(f"\n✓ All visualizations saved to: {viz_dir}/")
    print("=" * 60)

else:
    print("\n⚠️ Insufficient data for visualizations")


CREATING VISUALIZATIONS
✓ Created: success_by_tech_level.png
✓ Created: top_issues.png
✓ Created: severity_distribution.png
✓ Created: completion_time_distribution.png

✓ All visualizations saved to: visualizations_output/


In [11]:
# ============================================================
# CELL 7: GENERATE EXECUTIVE REPORT
# ============================================================

print("\n" + "=" * 60)
print("GENERATING EXECUTIVE REPORT")
print("=" * 60)

if test_results and personas:
    
    # Create comprehensive report
    executive_report = {
        'report_metadata': {
            'generated_at': datetime.now().isoformat(),
            'project_name': 'Tandem Test - AI-Powered UX Research',
            'report_type': 'Comprehensive Usability Analysis'
        },
        'executive_summary': {
            'total_personas': len(personas),
            'total_scenarios': len(test_scenarios),
            'total_tests_run': len(test_results),
            'overall_success_rate': f"{(sum(1 for r in test_results if r['task_completed'])/len(test_results)*100):.1f}%",
            'critical_issues_identified': len([i for i in critical_issues if i['priority'] == 'HIGH']),
            'avg_completion_time': f"{np.mean([r['time_seconds'] for r in test_results])/60:.1f} minutes"
        },
        'detailed_findings': {
            'success_by_tech_level': tech_analysis,
            'critical_issues': critical_issues[:10],
            'diversity_metrics': diversity_metrics,
            'roi_analysis': roi
        },
        'recommendations': [
            {
                'priority': 'HIGH',
                'category': 'Navigation',
                'issue': critical_issues[0]['issue'] if critical_issues else 'N/A',
                'recommendation': 'Simplify navigation structure and improve visual hierarchy',
                'impact': 'Will improve success rate by estimated 15-20%'
            },
            {
                'priority': 'HIGH',
                'category': 'Accessibility',
                'issue': 'Low proficiency users struggled significantly',
                'recommendation': 'Add contextual help and tooltips throughout interface',
                'impact': 'Will reduce completion time by estimated 20-30%'
            },
            {
                'priority': 'MEDIUM',
                'category': 'Error Handling',
                'issue': 'Error messages unclear or unhelpful',
                'recommendation': 'Revise error messages to be more actionable and user-friendly',
                'impact': 'Will reduce task abandonment by estimated 10-15%'
            }
        ]
    }
    
    # Save executive report
    report_dir = Path("./final_reports")
    report_dir.mkdir(exist_ok=True)
    
    report_file = report_dir / "executive_report.json"
    with open(report_file, 'w') as f:
        json.dump(executive_report, f, indent=2)
    
    print(f"✓ Executive report saved to: {report_file}")
    
    # Create human-readable summary
    summary_file = report_dir / "executive_summary.txt"
    with open(summary_file, 'w') as f:
        f.write("="*70 + "\n")
        f.write("TANDEM TEST - EXECUTIVE SUMMARY\n")
        f.write("AI-Powered UX Research System\n")
        f.write("="*70 + "\n\n")
        
        f.write("PROJECT OVERVIEW\n")
        f.write("-" * 70 + "\n")
        f.write(f"Testing Completed: {datetime.now().strftime('%B %d, %Y')}\n")
        f.write(f"Personas Tested: {len(personas)}\n")
        f.write(f"Test Scenarios: {len(test_scenarios)}\n")
        f.write(f"Total Tests Executed: {len(test_results)}\n\n")
        
        f.write("KEY FINDINGS\n")
        f.write("-" * 70 + "\n")
        summary = executive_report['executive_summary']
        f.write(f"Overall Success Rate: {summary['overall_success_rate']}\n")
        f.write(f"Critical Issues Found: {summary['critical_issues_identified']}\n")
        f.write(f"Average Completion Time: {summary['avg_completion_time']}\n\n")
        
        f.write("TOP 3 CRITICAL ISSUES\n")
        f.write("-" * 70 + "\n")
        for i, issue in enumerate(critical_issues[:3], 1):
            f.write(f"{i}. {issue['issue']}\n")
            f.write(f"   Impact: {issue['impact_percentage']:.1f}% of tests\n")
            f.write(f"   Priority: {issue['priority']}\n\n")
        
        f.write("ROI ANALYSIS\n")
        f.write("-" * 70 + "\n")
        f.write(f"Time Savings: {roi['savings']['time_hours']:.1f} hours ")
        f.write(f"({roi['savings']['time_percentage']:.1f}%)\n")
        f.write(f"Cost Savings: ${roi['savings']['cost_usd']:,.2f} ")
        f.write(f"({roi['savings']['cost_percentage']:.1f}%)\n\n")
        
        f.write("RECOMMENDATIONS\n")
        f.write("-" * 70 + "\n")
        for i, rec in enumerate(executive_report['recommendations'], 1):
            f.write(f"{i}. [{rec['priority']}] {rec['category']}\n")
            f.write(f"   Issue: {rec['issue']}\n")
            f.write(f"   Recommendation: {rec['recommendation']}\n")
            f.write(f"   Expected Impact: {rec['impact']}\n\n")
    
    print(f"✓ Executive summary saved to: {summary_file}")
    
    print("\n" + "=" * 60)
    print("✓ COMPREHENSIVE ANALYSIS COMPLETE!")
    print("=" * 60)
    print(f"\nAll reports saved in: {report_dir}/")
    print(f"Visualizations saved in: {viz_dir}/")
    print("\nProject deliverables:")
    print("  ✓ 30 diverse AI personas generated")
    print(f"  ✓ {len(test_scenarios)} test scenarios created")
    print(f"  ✓ {len(test_results)} usability tests executed")
    print("  ✓ Comprehensive analysis completed")
    print("  ✓ Executive reports generated")
    print("  ✓ Visualizations created")
    print("\n" + "="*70)
    print("TANDEM TEST PROJECT COMPLETE!")
    print("="*70)

else:
    print("\n❌ Insufficient data for report generation")


GENERATING EXECUTIVE REPORT
✓ Executive report saved to: final_reports/executive_report.json
✓ Executive summary saved to: final_reports/executive_summary.txt

✓ COMPREHENSIVE ANALYSIS COMPLETE!

All reports saved in: final_reports/
Visualizations saved in: visualizations_output/

Project deliverables:
  ✓ 30 diverse AI personas generated
  ✓ 150 test scenarios created
  ✓ 50 usability tests executed
  ✓ Comprehensive analysis completed
  ✓ Executive reports generated
  ✓ Visualizations created

TANDEM TEST PROJECT COMPLETE!
